### Implements German to English Translation using attention based LSTM [Bahdanau, 2014](https://arxiv.org/abs/1409.0473) in pyTorch

Additional notes and To do:
1. Code has been tried out on my own machine but need to run on GPU for larger training dataset
2. Add graphics for attention
3. Add BLEU score implementation

In [338]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [339]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [340]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('/Users/kalpeshpatel/data/iwslt-deen-prepared/train.de-en', encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [l.split(sep = ' ||| ') for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [341]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


In [342]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am", "i 'm",
    "he is", "he 's ",
    "she is", "she 's",
    "you are", "you re ",
    "we are", "we 're ",
    "they are", "they 're "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)
#def filterPair(p):
#    return p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [343]:
input_lang, output_lang, pairs = prepareData('de','eng')
vocab_input = input_lang.n_words
vocab_output = output_lang.n_words
print(random.choice(pairs))

Reading lines...
Read 179643 sentence pairs
Trimmed to 1025 sentence pairs
Counting words...
Counted words:
de 1751
eng 1527
['wir sind hauptfiguren in unseren eigenen leben .', 'we are the subjects of our own lives .']


In [381]:
class EncoderRNN(nn.Module):
    #input_size: Max index value for the input vocubulary
    #embedding_size: Dimension of embedding vector
    #hidden_size: hidden size for lstm. 
    def __init__(self, input_size, embedding_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.lstm = nn.LSTM(input_size = self.embedding_size, hidden_size = self.hidden_size, bidirectional = True)

    def forward(self, input):
        embedded = self.embedding(input)
        output = embedded
        output, (hidden,cell) = self.lstm(output)
        return output, hidden
    def initHidden(self):
        return torch.zeros(2, 1, self.hidden_size, device=device)

In [345]:
test = EncoderRNN(input_size = 10,hidden_size = 20, embedding_size = 13)

In [346]:
input = torch.randint(size = (11, 16),high = 10,dtype = torch.long)

In [347]:
out,hidden = test.forward(input)

In [348]:
out.shape

torch.Size([11, 16, 40])

In [370]:
#Params for init():
#output_size: Max index value for the output vocubulary. 
#embedding_size: Dimension of embedding vector
#hidden_size: hidden size for lstm. Let's assume we are using same hidden size for encoder and decoder

#Params for feedforward method:
#input_idx: Index of a word of the output or target sequence
#prev_decoder_output: tuple of previous (hidden state, cell memory) for decoder LSTM. hidden_state size: (1,batch_size,no of hidden units)
#encoder_output: output from the encoder of size: (Tx, batch_size,2*number of hidden untis). 
# Tx is length of input sentance


class AttnDecoderRNN(nn.Module):
    def __init__(self, output_size, hidden_size, embedding_size,dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        #No of hidden units in attension network
        self.hidden_size_att = 10
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.embedding_size = embedding_size
        self.dropout_p = dropout_p
        self.embedding = nn.Embedding(output_size, embedding_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.dense1 = nn.Linear(in_features = 2*hidden_size,out_features = self.hidden_size_att)
        self.dense2 = nn.Linear(in_features = hidden_size,out_features = self.hidden_size_att)
        self.dense3 = nn.Linear(in_features = self.hidden_size_att,out_features = 1)
        self.lstm = nn.LSTM(input_size = (self.embedding_size + 2*self.hidden_size ), hidden_size = self.hidden_size)
        #add output of dense1 and dense2. Pass it through tanh
        #apply dense3 to output of tanh. 
        #apply softmax to output of dense3.
        self.final_dense = nn.Linear(in_features = self.hidden_size, out_features = self.output_size)
        
    def forward(self,input_idx, prev_decoder_output, encoder_output):
        self.input_idx = input_idx
        self.prev_decoder_hidden = prev_decoder_output[0]
    
        embedding_output = self.embedding(self.input_idx)
        embeddeding_output = self.dropout(embedding_output)
        output1 = self.dense1(encoder_output)
        output2 = self.dense2(self.prev_decoder_hidden)
        output = torch.add(output1,1,output2)
        output = F.tanh(output)
        output = self.dense3(output)
        # I have selected dim = 0. Not sure if this is correct. 
        output = F.softmax(output,dim = 0)
        output = torch.mul(output,encoder_output)
        output = torch.sum(output,dim = 0,keepdim = True)
        output = torch.cat((output,embedding_output), dim = 2)
        #print("Final output dim:", output.shape)
        
        output,(hn,cn) = self.lstm(output,prev_decoder_output)
        # apply softmax along batch dimension. NOt sure if this is correct. 
        # out_pred has dimension of (# of layers, batch_size,size of hidden layer). Therefore, send 0th index to work with cross entropy.
        out_pred = F.log_softmax(self.final_dense(output),dim = 2)
        return out_pred[0],output,(hn,cn)
        

In [371]:
test_decoder = AttnDecoderRNN(output_size = 21,hidden_size = 20,embedding_size = 15)

In [372]:
input_idx = torch.randint(size = (1,16),high = 21,dtype = torch.long)

In [373]:
prev_decoder_hidden = torch.randn(size = (1,16,20))
prev_decoder_cell = torch.randn(size = (1,16,20))
prev_decoder_output = (prev_decoder_hidden,prev_decoder_cell)

In [374]:
pred,out1,out2 = test_decoder.forward(input_idx, prev_decoder_output,out)

In [375]:
pred.shape

torch.Size([16, 21])

In [376]:
# note: we are using batch size of 1
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [382]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    loss = 0
    
    output_encoder,hidden_encoder = encoder.forward(input_tensor)
    #now let's initialize hidden state for decoder from output of last layer of encoder
    decoder_hidden = hidden_encoder[1].view(1,hidden_encoder.size(1),hidden_encoder.size(2))
    decoder_c = torch.zeros_like(decoder_hidden)
    prev_decoder = (decoder_hidden,decoder_c)
    #print(init_hidden_decoder.shape)
    decoder_input = torch.tensor([[SOS_token]], device=device,dtype = torch.long)
    for di in range(target_length):
        out_pred,output, prev_decoder_1 = decoder.forward(
                decoder_input, prev_decoder, output_encoder)
        prev_decoder = prev_decoder_1
        loss += criterion(out_pred,target_tensor[di])
        decoder_input = torch.tensor([[target_tensor[di]]],dtype = torch.long) # Teacher forcing
        #print("Loss:", loss)

    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    return(loss.item() / target_length)

In [383]:
hidden_size = 256
embedding_size = 256
def trainIters(n_iters, learning_rate=0.01):
    print_loss_total = 0
    print_every = 100
    encoder = EncoderRNN(input_lang.n_words,hidden_size,embedding_size)
    decoder = AttnDecoderRNN(output_lang.n_words,hidden_size,embedding_size)
    
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        #tensors for one sentance
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('(%d %d%%) %.4f' % (iter, iter / n_iters * 100, print_loss_avg))


In [ ]:
trainIters(5000)